<a href="https://colab.research.google.com/github/vishnudas08/Small-AI-Tool-for-Text-Classification/blob/main/p2_text_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers
import torch
import torch.nn as nn
import pandas as pd
!pip install --upgrade datasets  # Ensure latest version

!pip install datasets transformers
import os
os.environ["HF_DATASETS_CACHE"] = "/content/hf_cache"

In [ ]:
from datasets import load_dataset
dataset= load_dataset("ag_news", download_mode="force_redownload")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
label_info= dataset['test'].features['label']
num_classes = label_info.num_classes
num_classes

4

In [ ]:
train_data= dataset['train']
train_data.features

{'text': Value('string'),
 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'])}

In [ ]:
test_data= dataset['test']

In [ ]:
train_df= pd.DataFrame(train_data)
test_df=pd.DataFrame(test_data)

In [ ]:
min_samples_per_class = train_df['label'].value_counts().min()
print(min_samples_per_class)


30000


In [ ]:
samples_per_class = min(min_samples_per_class, 807)

In [ ]:
from sklearn.utils import resample

train_df_reduced = (train_df
                    .groupby('label', group_keys=False)
                    .apply(lambda x: resample(x, replace=False, n_samples=samples_per_class, random_state=42))
                    .reset_index(drop=True))
train_df_reduced

/tmp/ipython-input-361373464.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: resample(x, replace=False, n_samples=samples_per_class, random_state=42))


,text,label
0,"Explosion Rocks Baghdad Neighborhood BAGHDAD, ...",0
1,BBC reporters' log BBC correspondents record e...,0
2,Israel welcomes Rice nomination; Palestinians ...,0
3,Medical Journal Calls for a New Drug Watchdog ...,0
4,Militants Kidnap Relatives of Iraqi Minister-T...,0
...,...,...
3223,"Enter your e-mail: According to wire reports, ...",3
3224,Video: Taking It with You (Ziff Davis) Ziff Da...,3
3225,European Winters Could Disappear by 2080 - Rep...,3
3226,Conjunction Junction for the Mars Rovers In th...,3


In [ ]:
train_df_reduced

,text,label
0,"Explosion Rocks Baghdad Neighborhood BAGHDAD, ...",0
1,BBC reporters' log BBC correspondents record e...,0
2,Israel welcomes Rice nomination; Palestinians ...,0
3,Medical Journal Calls for a New Drug Watchdog ...,0
4,Militants Kidnap Relatives of Iraqi Minister-T...,0
...,...,...
3223,"Enter your e-mail: According to wire reports, ...",3
3224,Video: Taking It with You (Ziff Davis) Ziff Da...,3
3225,European Winters Could Disappear by 2080 - Rep...,3
3226,Conjunction Junction for the Mars Rovers In th...,3


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df_reduced.text, train_df_reduced.label, test_size=0.2, random_state=0)

In [ ]:
len(X_train), len(y_test)

(2582, 646)

In [ ]:
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenizer_fn(text, label):
  inputs= tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
  return torch.tensor(inputs['input_ids']), torch.tensor(inputs['attention_mask']), torch.tensor(label, dtype=torch.long)
print(tokenizer_fn(['manu bhaker won the two olympic gold medals'] ,[0,1,2,3]))

(tensor([[  101,  2158,  2226,  1038, 20459,  2121,  2180,  1996,  2048,  4386,
          2751,  6665,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 

/tmp/ipython-input-3639650617.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(inputs['input_ids']), torch.tensor(inputs['attention_mask']), torch.tensor(label, dtype=torch.long)


In [ ]:
train_input_ids,  train_attention_mask, train_labels  =tokenizer_fn(X_train.values.tolist(), y_train.tolist())
test_inputs_ids, test_attention_mask, test_labels = tokenizer_fn(X_test.values.tolist(), y_test.tolist())

/tmp/ipython-input-3639650617.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(inputs['input_ids']), torch.tensor(inputs['attention_mask']), torch.tensor(label, dtype=torch.long)


In [ ]:
train_dataset= torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset= torch.utils.data.TensorDataset(test_inputs_ids, test_attention_mask, test_labels)

In [ ]:
train_dataLoader=DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataLoader=DataLoader(val_dataset, batch_size=128, shuffle=False)

In [ ]:
bert= BertModel.from_pretrained('bert-base-uncased')
bert.config.hidden_size

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

768

In [ ]:
class textClassifier(nn.Module):
  def __init__(self, num_classes):
    super(). __init__()
    self.bert= BertModel.from_pretrained('bert-base-uncased')

    for param in self.bert.parameters():
      param.requires_grad=False
    self.classifier= nn.Sequential(
        nn.Linear(self.bert.config.hidden_size, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256,num_classes)



    )
  def forward(self,input_ids, attention_mask):
    bert_output= self.bert(input_ids=input_ids,attention_mask=attention_mask)
    sentence_embedding= bert_output.last_hidden_state[:,0,:]
    return self.classifier(sentence_embedding)

In [ ]:
import torch.optim as optim

In [ ]:
model= textClassifier(num_classes=4).to(device)
optimizer= optim.Adam(model.parameters(), lr=0.001)
criterion= nn.CrossEntropyLoss()
#print(model)

In [ ]:
epochs=1
for epoch in range(epochs):
  model.train()
  total_train_loss=0

  for batch, (input_ids, attention_mask, labels) in enumerate(train_dataLoader):
    input_ids, attention_mask, labels= input_ids.to(device), attention_mask.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs= model(input_ids, attention_mask).squeeze()
    loss= criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f"Batch {batch}, epoch {epoch}, loss {loss.item():0.2f}")
    total_train_loss+= loss.item()
avg_train_loss= total_train_loss/len(train_dataLoader)
print(f" epoch {epoch +1}/{epochs}, Training loss : {avg_train_loss}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batch 0, epoch 0, loss 1.41
Batch 1, epoch 0, loss 1.31
Batch 2, epoch 0, loss 1.27
Batch 3, epoch 0, loss 1.22
Batch 4, epoch 0, loss 1.10
Batch 5, epoch 0, loss 1.04
Batch 6, epoch 0, loss 0.96
Batch 7, epoch 0, loss 0.91
Batch 8, epoch 0, loss 0.89
Batch 9, epoch 0, loss 0.76
Batch 10, epoch 0, loss 0.78
Batch 11, epoch 0, loss 0.81
Batch 12, epoch 0, loss 0.64
Batch 13, epoch 0, loss 0.71
Batch 14, epoch 0, loss 0.68
Batch 15, epoch 0, loss 0.68
Batch 16, epoch 0, loss 0.62
Batch 17, epoch 0, loss 0.60
Batch 18, epoch 0, loss 0.55
Batch 19, epoch 0, loss 0.54
Batch 20, epoch 0, loss 0.53
 epoch 1/1, Training loss : 0.8580218326477778


In [ ]:
model.eval()
total_val_loss=0
correct_predicted=0

with torch.no_grad():
  for input_ids, attention_mask, labels in val_dataLoader:
    input_ids, attention_mask, labels= input_ids.to(device), attention_mask.to(device), labels.to(device)
    outputs= model(input_ids, attention_mask).squeeze()
    loss= criterion(outputs, labels)
    total_val_loss += loss.item()
    pred= torch.argmax(outputs, dim=1)
    correct_predicted= torch.sum(pred == labels)
avg_val_loss= total_val_loss/ len(val_dataLoader)
val_accuracy= correct_predicted/ len(val_dataset)
print(f"validation loss{avg_val_loss},  val accuracy {val_accuracy}")

validation loss0.528363565603892,  val accuracy 0.007739937864243984


In [ ]:
label_map = dataset['train'].features['label'].int2str

In [ ]:
para="""The 1936 Berlin Olympics stands as one of the most iconic moments in Indian sports history. Under the captaincy of Dhyan Chand, the Indian men’s hockey team delivered a masterclass in skill and dominance, defeating Germany 8–1 in the final — all in front of a massive crowd, including Adolf Hitler himself.

Held in Nazi Germany, the Games were used as a propaganda tool by Hitler to showcase Aryan supremacy. However, India’s resounding victory disrupted that narrative. Dhyan Chand, often called the "Wizard of Hockey," scored three goals in the final, mesmerizing the crowd with his dribbling and control.

Legend has it that Hitler was so impressed by Dhyan Chand’s performance that he offered him a high-ranking position in the German army — which Dhyan Chand politely declined.

This match wasn't just a sporting win — it was a symbolic moment of pride for a colonized India, proving its excellence on a global stage during one of the most politically charged Olympics in history."""
para


'The 1936 Berlin Olympics stands as one of the most iconic moments in Indian sports history. Under the captaincy of Dhyan Chand, the Indian men’s hockey team delivered a masterclass in skill and dominance, defeating Germany 8–1 in the final — all in front of a massive crowd, including Adolf Hitler himself.\n\nHeld in Nazi Germany, the Games were used as a propaganda tool by Hitler to showcase Aryan supremacy. However, India’s resounding victory disrupted that narrative. Dhyan Chand, often called the "Wizard of Hockey," scored three goals in the final, mesmerizing the crowd with his dribbling and control.\n\nLegend has it that Hitler was so impressed by Dhyan Chand’s performance that he offered him a high-ranking position in the German army — which Dhyan Chand politely declined.\n\nThis match wasn\'t just a sporting win — it was a symbolic moment of pride for a colonized India, proving its excellence on a global stage during one of the most politically charged Olympics in history.'

In [ ]:
def predict(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=128)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask).float()
        predicted_class = torch.argmax(outputs, dim=1).item()
        return label_map(predicted_class)

#print(predict("manu bhaker won the two olympic gold medals"))
#print(predict("vishnu has discover tech solution for coolent in data centers"))
print(predict(para))

Sports


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
